## The Culture of International Relations - Text Analysis
### <span style='color: green'>SETUP </span> Prepare and Setup Notebook <span style='float: right; color: red'>MANDATORY</span>

## <span style='color: green'>PREPARE </span> Load and Prepare Corpus <span style='float: right; color: red'>MANDATORY</span>


In [ ]:
# Setup
%load_ext autoreload
%autoreload 2

import sys, os #, collections, zipfile
#import re, typing.re

sys.path = list(set(['.', '..']) - set(sys.path)) + sys.path

import nltk, textacy, spacy 
import pandas as pd
import ipywidgets as widgets
import bokeh, bokeh.plotting, bokeh.models, matplotlib.pyplot as plt
import common.utility as utility
import common.widgets_utility as widgets_utility
import common.widgets_config as widgets_config
import common.config as config
import common.utility as utility
import text_corpus
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

from beakerx.object import beakerx
from beakerx import *
from IPython.display import display, set_matplotlib_formats

logger = utility.getLogger('corpus_text_analysis')

import textacy_corpus_utility as textacy_utility
import domain_logic_vatican as domain_logic

utility.setup_default_pd_display(pd)

DATA_FOLDER, PATTERN = '../../data',  '*.txt'
DF_TAGSET = pd.read_csv(os.path.join(DATA_FOLDER, 'tagset.csv'), sep='\t').fillna('')

%matplotlib inline

# set_matplotlib_formats('svg')   
bokeh.plotting.output_notebook()

current_corpus_container = lambda: textacy_utility.CorpusContainer.container()
current_corpus = lambda: textacy_utility.CorpusContainer.corpus()


## <span style='color: green'>PREPARE </span> Load and Prepare Corpus <span style='float: right; color: red'>MANDATORY</span>


In [ ]:
import textacy_corpus_utility as textacy_utility
import textacy_corpus_gui

try:
    container = current_corpus_container()
    # FIXME VARYING ASPECTS: document_index = WTI_INDEX for tCoIR
    textacy_corpus_gui.display_corpus_load_gui(DATA_FOLDER, document_index=None, container=container, compute_ner=True, domain_logic=domain_logic)
except Exception as ex:
    raise
    logger.error(ex)

## <span style='color: green;'>MODEL</span> Display Named Entities<span style='color: green; float: right'>TRY IT</span>
Spacy NER, note that "ner" must be enabled in corpus pipeline.

In [ ]:
# Display Named Entities
import gui_utility
from spacy import displacy

def display_document_entities_gui(corpus, document_index):
    
    # FIXME VARYING ASPECT: Add "document_name" to document_index, or function that creates name
    filenames = document_index.filename
    document_options = list(sorted(zip(filenames,filenames.index), key=lambda x: x[0]))

    gui = types.SimpleNamespace(
        position=1,
        output=widgets.Output(layout={'border': '1px solid black'}),
        document_id=widgets.Dropdown(description='Document', options=document_options, value=document_options[1][1], layout=widgets.Layout(width='50%')),
        left=widgets.Button(description='<<', button_style='Success', layout=widgets.Layout(width='40px')),
        right=widgets.Button(description='>>', button_style='Success', layout=widgets.Layout(width='40px')),
    )

    def display_document_entities(corpus, document_id):
        gui.output.clear_output()
        with gui.output:        
            doc = textacy_utility.get_document_by_id(corpus, document_id)
            displacy.render(doc, style='ent', jupyter=True)
    
    def back_handler(*args):
        if gui.position == 0:
            return
        gui.output.clear_output()
        gui.position = (gui.position - 1) % len(document_options)
        gui.document_id.value = document_options[gui.position][1]
        #itw.update()
        
    def forward_handler(*args):
        gui.output.clear_output()
        gui.position = (gui.position + 1) % len(document_options)
        gui.document_id.value = document_options[gui.position][1]
    
    gui.left.on_click(back_handler)
    gui.right.on_click(forward_handler)
    
    display(widgets.VBox([
        widgets.HBox([gui.document_id, gui.left, gui.right]),
        widgets.VBox([gui.output], layout=widgets.Layout(margin_top='20px', height='600px',width='100%'))
    ]))
    
    itw = widgets.interactive(
        display_document_entities,
        corpus=widgets.fixed(corpus),
        document_id=gui.document_id
    )
    
    itw.update()
    
try:
    corpus = current_corpus()
    document_index = domain_logic.compile_documents(corpus)
    display_document_entities_gui(corpus, document_index=document_index)
except Exception as ex:
    logger.error(ex)


In [ ]:
def extract_entities(doc, include_types=None, drop_determiners=True):
    
    entities = (x for x in doc.ents if not x.text.isspace())
    
    if include_types is not None:
        assert isinstance(include_types, (set, list, tuple))
        entities = (x for x in entities if x.label_ in include_types)

    if drop_determiners is True:
        entities = (x if x[0].pos != DET else SpacySpan(x.doc, x.start + 1, x.end, label=x.label, vector=x.vector) for x in entities)

    for x in entities:
        yield x
    


## <span style='color: green;'>MODEL</span> Extract Named Entities<span style='color: green; float: right'>TRY IT</span>
Spacy NER, note that "ner" must be enabled in corpus pipeline.

In [ ]:
#%%file run_ner_places.py
#from cytoolz import itertoolz
import textacy_corpus_utility as textacy_utility
import spacy
import pandas as pd
import domain_logic_vatican as domain_logic
import text_corpus
import logging
import string

from spacy.tokens.span import Span as SpacySpan

logger = logging.getLogger('ner')
logger.setLevel(logging.INFO)

NON_PRINT_CHARS = set([chr(i) for i in range(128)]).difference(string.printable)

DEL_WS_CHARS = str.maketrans('', '', '"\n\t')
DEL_CRAP_CHARS = str.maketrans('', '', '"\n\t?@')# + NON_PRINT_CHARS)
#DEL_NON_PRINT = str.maketrans('', '', NON_PRINT_CHARS)

#def get_doc_places(doc):
#    return ( (w.text, w.lemma_, str(w.lemma_).translate(DEL_CRAP_CHARS), w[0].ent_type_)
#                for w in doc.ents if w[0].ent_type_ in ['LOC', 'GPE'] and w.lemma_.strip() != '' )

def get_doc_place_entities(doc, drop_determiner=True):
    
    DET = spacy.parts_of_speech.DET
    PUNCT = spacy.parts_of_speech.PUNCT
    
    include_types = ['LOC', 'GPE']
    
    entities = doc.ents
    entities = (x for x in entities if not x.text.isspace())
    entities = (x for x in entities if x.lemma_.strip() != '')
    entities = (x for x in entities if x.label_ in include_types)

    if drop_determiner is True:
        entities = (x if x[0].pos != DET else SpacySpan(x.doc, x.start + 1, x.end, label=x.label, vector=x.vector) for x in entities)
        
    for x in entities:
        text = x.text.translate(DEL_WS_CHARS)
        lemma = x.lemma_.translate(DEL_WS_CHARS)
        #tidy_lemma = lemma.translate(DEL_CRAP_CHARS).replace('  ', ' ')
        tidy_lemma = lemma.replace('  ', ' ')
        yield (text, lemma, tidy_lemma, x[0].ent_type_)
        
#def get_corpus_places(corpus):
#    return itertoolz.chain.from_iterable(( get_doc_places(doc) for doc in corpus ))

def create_source_stream(source_path, lang, document_index=None):
    reader = text_corpus.CompressedFileReader(source_path)
    stream = domain_logic.get_document_stream(reader, lang, document_index=document_index)
    return stream

def create_nlp(model='en_core_web_sm', disable=None):
    nlp = spacy.load(model, disable=disable)
    nlp.tokenizer = textacy_utility.keep_hyphen_tokenizer(nlp)
    return nlp

source_paths = [ '../../data/benedict-xvi_curated_20190326.txt_preprocessed.zip', '../../data/francis_curated_20190326.txt_preprocessed.zip' ]

model_name = 'en_core_web_lg'

logger.info('Loading model {}'.format(model_name))
nlp = create_nlp(model=model_name, disable=('parser', 'textcat'))

for source_path in source_paths:
    logger.info('Processing {}...'.format(source_path))
    stream = create_source_stream(source_path, 'en')
    file_counter = 0
    places = []
    for filename, text, _ in stream:
        file_counter += 1
        doc = nlp(text)
        places.extend(list(get_doc_place_entities(doc)))
        if file_counter % 100 == 0:
            logger.info('Processed {} files...{} places found...'.format(file_counter, len(places)))
            #break
        doc = None
        
df = pd.DataFrame(places, columns=['text', 'lemma', 'tidy_lemma', 'ent_type'])
df.to_csv('./NER_with_tagging_total.txt', sep='\t')

df_grouped = df.groupby(['tidy_lemma', 'ent_type']).size().reset_index()

df_grouped.to_csv('./NER_with_tagging_total_tidy_lemma_grouped.txt', sep='\t')


#[z for z in textacy.extract.entities(doc)]
#[[ ent for ent in textacy.extract.entities(doc) ] for doc in corpus if len(doc.ents or []) > 0 ]


In [ ]:
%%bash
nohup python3 run_ner_places.py &


## <span style='color: green;'>DESCRIBE</span> Display Named Entity Statistics<span style='color: green; float: right'>TRY IT</span>
Spacy NER, note that "ner" must be enabled in corpus pipeline.

In [ ]:

def compile_named_entity_data(corpus, document_index, drop_determiners=True, min_freq=1):
    #textacy.extract.entities(doc, include_types=None, exclude_types=None, drop_determiners=True, min_freq=1)
    data = [[
        (doc._.meta['document_id'], ent[0].ent_type_, ent.text, ent.lemma_)
             for ent in textacy.extract.entities(doc, exclude_types=('CARDINAL',), drop_determiners=drop_determiners, min_freq=min_freq) ]
                for doc in corpus
    ]
    data = utility.flatten(data)
    df = pd.DataFrame(data, columns=['document_id', 'ent_type', 'text', 'lemma']).set_index('document_id')
    df = pd.merge(df, document_index, left_index=True, right_index=True, how='inner')
    return df[df.year > 0][['pope', 'year', 'genre', 'ent_type', 'text', 'lemma', 'filename']].reset_index()

def display_grouped_by_entities_gui(corpus, document_index):
    
    columns = compile_named_entity_data([corpus[0]], document_index).columns
    
    group_by_options = [ (x.title(), x) for x in columns if x not in [ 'ent_type', 'text', 'lemma', 'filename', 'index'] ]
    group_by_values = [ x for _, x in group_by_options ]
    gui = types.SimpleNamespace(
        group_by=widgets.SelectMultiple(description='Group by', options=group_by_options, value=group_by_values, rows=3, layout=widgets.Layout(width='180px')),
        filter_punct=widgets.ToggleButton(value=False, description='Filter punct',  tooltip='Filter out punctuations', icon='check'),
        drop_determiners=widgets.ToggleButton(value=True, description='Drop DET',  tooltip='Drop_determiners`', icon='check'),
        output=widgets.Output(layout={'border': '1px solid black'}),
        min_freq=widgets.IntSlider(description='Min freq', min=1, max=10, value=2, step=1, layout=widgets.Layout(width='400px')),
    )
        
    def display_grouped_by_entities(corpus, group_by, drop_determiners, min_freq):
        gui.output.clear_output()
        named_entities = compile_named_entity_data(corpus, document_index, drop_determiners, min_freq)
        with gui.output:
            df = named_entities.groupby(list(group_by) + ['ent_type', 'lemma']).size().reset_index()
            df = df.rename(columns={0:'Count'})
            df = df.sort_values('Count', ascending=False)
            display(df)

    itw = widgets.interactive(
        display_grouped_by_entities,
        corpus=widgets.fixed(corpus),
        #named_entities=widgets.fixed(named_entities),
        group_by=gui.group_by,
        drop_determiners=gui.drop_determiners,
        min_freq=gui.min_freq
    )

    display(widgets.VBox([
        widgets.HBox([gui.group_by, gui.drop_determiners, gui.min_freq]),
        widgets.VBox([gui.output]),
        itw.children[-1]
    ]))
    
    
try:
    corpus = current_corpus()
    document_index = domain_logic.compile_documents(corpus)
    display_grouped_by_entities_gui(corpus, document_index)
except Exception as ex:
    raise
    logger.error(ex)
    

## <span style='color: green;'>MODEL</span> Stanford NER Tagger (CoreNLP)<span style='color: green; float: right'>SKIP</span>

### <span style='color: green;'>PREPARE</span> Verify that Stanford CoreNLP is up and running<span style='color: green; float: right'>SKIP</span>
Stanford CoreNLP server must be started as described in:  https://stanfordnlp.github.io/CoreNLP/corenlp-server.html

With docker:
```bash
docker pull frnkenstien/corenlp
docker run -p 9000:9000 --name coreNLP --rm -i -t frnkenstien/corenlp
```

In [ ]:
STANFORD_CORE_NLP_URL = 'http://localhost:9000'

try:
    from nltk.parse import corenlp
    corenlp_tagger = corenlp.CoreNLPParser(url=STANFORD_CORE_NLP_URL, encoding='utf8', tagtype='ner')
    input_tokens = 'Stony Brook University in NY'.split()
    tagged_output = corenlp_tagger.tag(input_tokens)
    print('Stanford tagger is up and running!')
    print(' Result: ' + ' '.join([ x + '/' + y for x,y in tagged_output]))
except: # (ConnectionError, ConnectionRefusedError):
    logger.error('Server not found! Please start Stanford CoreNLP Server!')
    

In [ ]:
%%file run_stanford_ner.py
import os, sys, time
import glob
import types
import ipywidgets as widgets
import text_corpus
import domain_logic_vatican as domain_logic
import nltk
import pandas as pd
import zipfile

sys.path = list(set(['.', '..']) - set(sys.path)) + sys.path

import common.widgets_config as widgets_config
import common.utility as utility

logger = utility.getLogger('corpus_text_analysis')

from nltk.parse import corenlp

STANFORD_CORE_NLP_URL = 'http://localhost:9000'

def merge_entities(entities):
    n_entities = len(entities)
    if n_entities <= 1:
        return entities
    merged = entities[:1]
    for doc_id, i_n, w_n, t_n in entities[1:]:
        doc_id_p, i_p, w_p, t_p = merged[-1]
        if i_n == i_p + 1 and t_n == t_p:
            merged[-1] = (doc_id, i_n, '_'.join([w_p, w_n]), t_p)
        else:
            merged.append((doc_id, i_n, w_n, t_n))
    return merged

def recognize_named_entities(tagger, doc_id, text, excludes=None, includes=None):
    ''' excludes not used '''
    sentences = nltk.sent_tokenize(text)
    start_index = 0
    excludes = excludes or []
    includes = includes or []
    merged_ents = []
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        tagged_data = tagger.tag(tokens)
        ents = [ (doc_id, start_index + index, word, ent_type) for index, (word, ent_type) \
                in enumerate(tagged_data) if ent_type in includes ]
        start_index += len(sentence)
        merged_ents.extend(merge_entities(ents))
    return merged_ents
           
def compute_stanford_ner(source_file, service_url=STANFORD_CORE_NLP_URL, excludes=None, includes=None):
    
    includes = includes or (
        'LOCATION', 'CITY', 'STATE_OR_PROVINCE', 'COUNTRY'
    )
    
    # For English, by default, this annotator recognizes named (PERSON, LOCATION, ORGANIZATION, MISC),
    # numerical (MONEY, NUMBER, ORDINAL, PERCENT), and temporal (DATE, TIME, DURATION, SET) entities (12 classes).
    # Adding the regexner annotator and using the supplied RegexNER pattern files adds support for the fine-grained and additional entity 
    # classes EMAIL, URL, CITY, STATE_OR_PROVINCE, COUNTRY, NATIONALITY, RELIGION, (job) 
    # TITLE, IDEOLOGY, CRIMINAL_CHARGE, CAUSE_OF_DEATH (11 classes) for a total of 23 classes.

    assert os.path.isfile(source_file), 'File missing!'
    
    tagger = corenlp.CoreNLPParser(url=service_url, encoding='utf8', tagtype='ner')
    #tokenizer = corenlp.CoreNLPParser(url=service_url, encoding='utf8')
    
    reader = text_corpus.CompressedFileReader(source_file)
    document_index = domain_logic.compile_documents_by_filename(reader.filenames)
    stream = domain_logic.get_document_stream(reader, 'en', document_index=document_index)
    
    i = 0
    ner_data = []
    for filename, text, metadata in stream:
        print(filename)
        document_id = document_index.loc[document_index.filename == filename, 'document_id'].values[0]
        ner = recognize_named_entities(tagger, document_id, text, excludes, includes)
        ner_data.extend(ner)
        i += 1
        if i % 10 == 0:
            logger.info('Processed {} files...'.format(i))
            # break
        
    return ner_data

def compute_and_store_stanford_ner(source_file):
    
    ner_data = compute_stanford_ner(source_file=source_file)

    df = pd.DataFrame(ner_data, columns=['doc_id', 'pos', 'entity', 'ent_type'])
    df.index.name = 'id'

    store_name = 'ner_{}_{}.txt'.format(
        os.path.splitext(os.path.split(source_file)[1])[0],
        time.strftime("%Y%m%d%H%M%S", time.localtime())
    )

    df.to_csv(store_name, sep='\t')
    logger.info('Result stored in %s', store_name)
    return df
        
def display_stanford_ner_gui(data_folder):
    
    lw = lambda w: widgets.Layout(width=w)
    
    corpus_files = sorted(glob.glob(os.path.join(data_folder, '*.txt.zip')))
    
    gui = types.SimpleNamespace(
        output=widgets.Output(layout={'border': '1px solid black'}),
        source_path=widgets_config.dropdown(description='Corpus', options=corpus_files, value=corpus_files[-1], layout=lw('300px')),
        compute=widgets.Button(description='Compute', button_style='Success', layout=lw('100px'))
    )
    
    display(widgets.VBox([
        widgets.HBox([
            widgets.VBox([
                gui.source_path,
            ]),
            widgets.VBox([
                gui.compute,
            ]),
        ]),
        gui.output
    ]))
    
    def compute_stanford_ner_callback(*_args):
        
        gui.output.clear_output()
        
        with gui.output:
            df = compute_and_store_stanford_ner(gui.source_path.value)
            display(df)
            
    gui.compute.on_click(compute_stanford_ner_callback)


#data_folder = '../../data'
#display_stanford_ner_gui(data_folder)
for source_file in [ '../../data/francis_curated_20190326.txt.zip', '../../data/benedict-xvi_curated_20190326.txt.zip', ]:
    compute_and_store_stanford_ner(source_file)
    

In [ ]:
def load_document_index(path):
    reader = text_corpus.CompressedFileReader(path)
    df = domain_logic.compile_documents_by_filename(reader.filenames)
    df['document_id'] = df.document_id.astype(np.int32)
    df = df.rename(columns={'document_id': 'doc_id'})
    df = df.set_index('doc_id')
    df = df[['pope', 'genre', 'year', 'filename']]
    return df

def load_ner_result(path):
    df = pd.read_csv(path, sep='\t')
    df['doc_id'] = df.doc_id.astype(np.int32)
    df = df.set_index('id')
    return df

def compile_ner_result(source_path, result_path):
    document_index = load_document_index(source_path)
    df_ner = load_ner_result(result_path)
    df_agg = df_ner.groupby(['doc_id', 'ent_type', 'entity']).size().reset_index().rename(columns={0: 'entity_count'})
    df_doc_agg = document_index.merge(df_agg, left_index=True, right_on='doc_id', how='inner')
    return df_doc_agg

df_francis = compile_ner_result('../../data/benedict-xvi_curated_20190326.txt.zip', 'ner_benedict-xvi_curated_20190326.txt_20190319193045.txt')
df_benedict = compile_ner_result('../../data/francis_curated_20190326.txt.zip', 'ner_francis_curated_20190326.txt_20190326145420.txt')

df_popes = pd.concat([df_francis, df_benedict], ignore_index=True, axis=0)

df_popes_entity_agg = df_popes.groupby(['pope', 'entity']).agg({'entity_count': 'sum'}).reset_index()
df_entity_agg = df_popes.groupby(['entity']).agg({'entity_count': 'sum'}).reset_index()

print(df_popes_entity_agg.entity_count.sum(), df_entity_agg.entity_count.sum())
    
with pd.ExcelWriter('ner_francis_and_benedict_stanford_20190326.xlsx') as writer:
    df_popes.to_excel(writer, sheet_name='Pivot (document)'),
    df_popes_entity_agg.to_excel(writer, sheet_name='Pivot (pope, entity)'),
    df_entity_agg.to_excel(writer, sheet_name='Pivot (entity)'),
    
    

In [ ]:
# df_ner[df_ner.doc_id == 3]
df_agg = df_ner.groupby(['doc_id', 'ent_type', 'entity']).size().reset_index().rename(columns={0: 'count'})
df_agg

df_pope_year_genre_type_agg = df_doc_agg.groupby(['pope', 'year', 'genre', 'ent_type', 'entity']).agg({'count': 'sum'}).reset_index()

